In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from collections import Counter
import re
from bs4 import BeautifulSoup

%matplotlib inline

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv',
                  encoding = 'latin-1')

In [3]:
data.dropna(axis = 1 ,inplace = True)

In [4]:
data.columns = ['label','message']

In [5]:
data['encoded_label'] = [1 if label == 'spam' else 0 for label in data.label]

In [190]:
class formating_message:
    
    def __init__(self,bs = False):
        self.bs = bs
        
    
    def word_cleaner(self,message):
        StopWords = stopwords.words('english')
        if self.bs:
            message = BeautifulSoup(message).get_text()
        no_punc = re.sub('[^A-Za-z ]','',message)
        no_punc = re.sub(' \s+','',no_punc)
        word_list = no_punc.split(' ')
        word_lower = list(map(str.lower,word_list))
        word_nost = [w for w in word_lower if (w not in StopWords and len(w) > 2)]
        mess_back = ' '.join(word_nost)
        
        return mess_back
    
    def tokenizer(self,message):
        word_token = []
        message_cleaned = self.word_cleaner(message)
        tokens = message_cleaned.split(' ')
        if (len(tokens) > 0):
            word_token.extend(tokens)
        return set(word_token)

In [267]:
cleaner = formating_message()

In [8]:
data.message[0:1][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [9]:
data.message[3]

'U dun say so early hor... U c already then say...'

In [10]:
for message in data.message[0:5]:
    print(cleaner.tokenizer(message))

{'wat', 'buffet', 'jurong', 'crazy', 'bugis', 'available', 'point', 'world', 'great', 'cine', 'amore', 'got'}
{'joking', 'oni', 'wif', 'lar'}
{'apply', 'tkts', 'toto', 'receive', 'ratetcs', 'comp', 'win', 'maytext', 'questionstd', 'overs', 'wkly', 'ina', 'free', 'txt', 'entry', 'final', 'cup'}
{'dun', 'hor', 'say', 'already', 'early'}
{'though', 'dont', 'think', 'nah', 'lives', 'around', 'goes', 'usf'}


In [11]:
data.index

RangeIndex(start=0, stop=5572, step=1)

In [293]:
class NaivesBayesClassifier:
    
    def __init__(self,Tokenizer,salut):
        self.Tokenizer = Tokenizer 
        self.salut = salut

        
    def fit(self,messages,labels):
        self.classes = np.unique(labels)
        self.n_classes = [] 
        
        n = len(messages)
        self.priors = []
        
        for label in self.classes:
            self.n_classes.append(np.sum(labels == label))
            
        for lab in self.classes:
            self.priors.append(self.n_classes[lab] / n)
        
        self.n = len(messages)
        total_word = []
        
        self.counter_list = [None] * len(self.classes)
        for lab in self.classes:
            self.counter_list[lab] = Counter()
            
        for lab in self.classes:
            for i in range(len(messages)):
                if (labels[i] == lab):
                    message_tokenized = self.Tokenizer.tokenizer(messages[i])
                    for word in message_tokenized:
                        if (word != ''):
                            total_word.append(word)
                            self.counter_list[lab][word] += 1
        self.unique_word = set(total_word) 
        
        for lab in self.classes:
            for word in self.unique_word:
                if word not in self.counter_list[lab].keys():
                    self.counter_list[lab][word] = 0
                
                
        return self
        
        
    def _word_prob(self,message_test):
        all_test_word = []
        
        for lab in self.classes:
            self.n_classes.append(len(self.counter_list[lab]))
            
        for message in message_test:
            mess_test_tok = self.Tokenizer.tokenizer(message)
            for word in mess_test_tok:
                all_test_word.append(word)
                
        unique_test_word = set(all_test_word)
        n_train = len(self.unique_word)
        word_not_in_train = [w for w in unique_test_word if w not in self.unique_word]
        full_word = list(self.unique_word) + word_not_in_train
        n_toadd = len(word_not_in_train)
        
        for lab in self.classes:
            for word in full_word:
                if word in word_not_in_train:
                    self.counter_list[lab][word] = 100000 *(1 / (len(self.unique_word)+ n_toadd)) # avoid infinity for very small value
                else:
                    self.counter_list[lab][word]  = 100000 *((self.counter_list[lab][word] + 1) / (len(self.unique_word) + n_toadd))
        
        return self
            
            
    def predict(self,message_test):
        self._word_prob(message_test)
        predict_classes = Counter()
        predicted_class = []

        for message in message_test:
            mess_test = self.Tokenizer.tokenizer(message)
            for lab in self.classes:
                predict_sum = 0
                for word in mess_test:
                    predict_sum += np.log(self.counter_list[lab][word]) # avoid log(0) don't know where it's coming from
                predict_classes[lab] = predict_sum + np.log(self.priors[lab])

            predicted_class.append(predict_classes.most_common(1)[0][0])

        return predicted_class                

In [205]:
new = list(NB.unique_word) + ['test']

In [206]:
'test' in new

True

In [207]:
from collections import Counter

In [208]:
np.sum((data.encoded_label == 1) )

747

In [277]:
NB = NaivesBayesClassifier(formating_message(),'Hey test')

In [278]:
NB.fit(data.message[0:3000],data.encoded_label[0:3000])

In [279]:
NB.priors

[0.8636666666666667, 0.13633333333333333]

In [280]:
predicted  = NB.predict(data.message[3000:])

In [281]:
np.mean(predicted == data.encoded_label[3000:]) #Super cool !~

0.9572317262830482

In [282]:
#Testing the algorithm on another dataset

In [266]:
train_imdb = pd.read_csv('labeledTrainData.tsv',delimiter = '\t')

In [232]:
test_imdb = pd.read_csv('testData.tsv',delimiter = '\t')

In [233]:
train_imdb.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [268]:
test_text = BeautifulSoup(train_imdb.review[0]).get_text()

In [294]:
NB_imdb = NaivesBayesClassifier(formating_message(bs = True),'Hey There :)')

In [295]:
NB_imdb.fit(train_imdb.review,train_imdb.sentiment)

In [296]:
NB_imdb.priors

[0.5, 0.5]

In [300]:
predicted_imdb = NB_imdb.predict(test_imdb.review)

In [312]:
count_classes(predicted_imdb)

0 : 12222
1 : 12778


In [311]:
def count_classes(liste):
    unique_class = np.unique(liste)
    for cat in unique_class:
        print(f'{cat} : {liste.count(cat)}')
    

In [307]:
df_prediction = pd.DataFrame({
    'id' : test_imdb.id,
    'sentiment' : predicted_imdb
})

In [308]:
df_prediction.to_csv('submission.csv',index = False)

In [313]:
#0.845 on kaggle :) 